In [1]:
import requests
import json
import pandas as pd
import random

'''

Get toxic score for the text from Hatescan API.
The API supports english language text.
'''
def hatescan_toxic_classifier(text):
    # adding \ infront of " in the text
    text = text.replace('"', '\\"')

    payload = '{"text": "' + text +'"}'

    json_payload =json.loads(payload, strict=False)

    headers={"Content-Type": "application/json; charset=utf-8"}

    # Hatescan API url
    api_hatescan_url = 'https://hatescan.dsv.su.se/predict'

    # sending post request to the API
    hatescan_response = requests.post(api_hatescan_url, headers=headers, json=json_payload)

    # return toxic probability
    return hatescan_response.json()

if __name__ == '__main__':
    # TODO
    '''
    1) write code to read the text file with data and store the results (from the file) in a list. Each post is an entry in the list. 
    2) loop through the list to get a post and send it to the function get_toxic_probability 
    3) The function get_toxic_probability returns the probability that a post (text) is toxic
    4) save the probability score of the post (text) together with the text in suitable format (e.g excel, tsv)
    '''

    _path = 'stormfront_2000.txt'
    # Initialize an empty list to store the file contents
     
    predict, post, toxic_flag = [],[],[]

    # Open the file in read mode
    try:
        with open(_path, 'r', encoding="utf-8") as file:
        # Read each line of the file and append it to the list
            for line in file:
                toxic_score = hatescan_toxic_classifier(line) 

                #print(toxic_score, line)
                score = list(toxic_score.values())[0]

                predict = predict + [score]

                '''
                Assiging an flag of "1" to a Post if it fall above the threshold, and "0" to Post if it fall below the thresh hold
                This will be used to compare with the hand annotation from the 4 group Members
                '''
                if float(score) >= 50:
                    _flag = 1
                else:
                    _flag = 0

                post = post + [line]
                toxic_flag = toxic_flag + [_flag]

        #Creating a dictionary out of the 2 lists for predicted score and Post
        content = { "Prediction":predict, "Toxic ?":toxic_flag, "Post":post}
        print(content)

        # Create a DataFrame from the dictionary
        df = pd.DataFrame(content)

        # Specify the Excel file name
        excel_file = 'prediction_2000_class.xlsx'

        # Write the DataFrame to an Excel file
        df.to_excel(excel_file, index=False)

        print(f'Data written to {excel_file}')

    except FileNotFoundError:
        print(f"File not found at path: {_path}")
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")

In [ ]:
# Store all non toxic labeled tweets 
df0 = df[df['Toxic ?'] == 0]

# Specify the Excel file name
excel_file = 'prediction_2000_non_toxic.xlsx'

# Write the DataFrame to an Excel file
df0.to_excel(excel_file, index=False)

# Store all toxic labeled tweets 
df1 = df[df['Toxic ?'] == 1]

# Specify the Excel file name
excel_file = 'prediction_2000_toxic.xlsx'

# Write the DataFrame to an Excel file
df1.to_excel(excel_file, index=False)

print(f'Data written has been written')


In [ ]:
'''
Creating Sample files for group annotation 
'''
Column_names = ['Suba', 'Johannes', 'Kunal', 'Joseph']

In [ ]:
'''
Choosing a Random sampls for both the toxic and non toxic for anotation
The the whole sample space of the toxic coimments is been considered for annotation
The sample from the non toxic is calculated using a confidence level of 85% with margin of error 5% 
and a population size of 1860, since we are considering thr whole predict sample of the toxic as a sample to be annotated manually
'''

# random toxic samples 

toxic_sample = df1

# Choosing random nontoxic samples
non_toxic_sample = df0.sample(n=187)

# Randonly Combining toxic and non toxic samples to give a total of annotable samples of 327

samples = pd.concat([df1, non_toxic_sample])
samples = samples.sample(frac=1).reset_index(drop=True)

# Add four empty columns before the last column
for i in range(4):
    samples.insert(len(samples.columns) - 1, Column_names[i], '')

# Creating the excel file
excel_file = 'annotate_sample.xlsx'

samples.to_excel(excel_file, index=False)
print(f'Data written to {excel_file}')